**Install requirements**

In [ ]:
%pip install -r requirements.txt

**Import libraries**

In [45]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

**Access website**

In [39]:
url = 'https://anitsayac.com/'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')

**Extract links of individual webpages**

In [40]:
spans = soup.find_all('span', class_='xxy')
links = []
for span in spans:
    link = span.find('a')
    links.append(link['href'])

**Function to extract data**

In [41]:
def get_data(links):
    fields = [
        'Ad Soyad:',
        'Maktülün yaşı: ',
        'İl/ilçe: ',
        'Tarih: ',
        'Neden öldürüldü:',
        'Kim tarafından öldürüldü:',
        'Korunma talebi:',
        'Öldürülme şekli:',
        'Failin durumu: '
    ]
    data_list = []
    
    for link in links:
        url = 'https://anitsayac.com/' + link
        details_page = requests.get(url).text
        local_soup = BeautifulSoup(details_page, 'html.parser')
        data = []
        
        for field in fields:
            tag = local_soup.find('b', string=field)
            if tag:
                data.append(tag.next_sibling.strip())
            else:
                data.append('')    
        data_list.append(data)

    df = pd.DataFrame(data_list,columns=fields)
    return df

**Extract data**

In [42]:
data = get_data(links)

**Rename columns**

In [43]:
data.columns =[
    'Ad',
    'Yaş',
    'Konum',
    'Tarih',
    'Öldürülme_sebebi',
    'Fail',
    'Korunma_talebi',
    'Öldürülme_şekli',
    'Failin_durumu'
]

data

,Ad,Yaş,Konum,Tarih,Öldürülme_sebebi,Fail,Korunma_talebi,Öldürülme_şekli,Failin_durumu
0,Sudenaz U.,Reşit,Diyarbakır,08/11/2024,Nefret Suçu,Tanımadığı Birileri,Yok,Kesici Alet,Tutuklu
1,Asiye Günay,Reşit,Kütahya,06/11/2024,Tespit Edilemeyen,Tespit Edilemeyen,Yok,Tespit Edilemeyen,Tespit Edilemeyen
2,Fahriye Asena Eskikanbur,Reşit,Adana,03/11/2024,Reddedilme,Eski Sevgilisi,Yok,Ateşli Silah,İntihara Teşebbüs
3,Elif Gözen,Reşit,Hatay,03/11/2024,Tespit Edilemeyen,Kocası,Yok,Tespit Edilemeyen,Tutuklu
4,Bahriye Kalaycı,Reşit,Konya,02/11/2024,Tartışma,Eski Kocası,Yok,Kesici Alet,İntihara Teşebbüs
...,...,...,...,...,...,...,...,...,...
5023,Gülceren Karakaş,,,10/01/2008,Tespit Edilemeyen,Tanımadığı Biri,Tespit Edilemeyen,Kesici Alet,
5024,Hashatun Danışmaz,,,09/01/2008,Boşanma,Kocası,Var,Ateşli Silah,
5025,Emel Çalış,,,07/01/2008,Reddetme,Eski Sevgilisi,Tespit Edilemeyen,Ateşli Silah,
5026,Gönül Özcan,,,06/01/2008,Kriz ve İşsizlik,Damadı,Tespit Edilemeyen,Ateşli Silah,


**Connect to mysql**

In [47]:
# Load credentials from config file
with open('db_config.json') as f:
    config = json.load(f)

db_user = config["DB_USER"]
db_pass = config["DB_PASSWORD"]
db_addr = config["DB_HOST"]
db_name = config["DB_NAME"]
engine = create_engine(f'mysql+mysqldb://{db_user}:{db_pass}@{db_addr}/{db_name}')

if not database_exists(engine.url): 
    create_database(engine.url)
else:
    engine.connect()

**Load data frame to database**

In [48]:
data.to_sql(con=engine, name='victims', if_exists='replace', index=False)


5028